In [58]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import mysql.connector
import json
from pandas.io.json import json_normalize
from config import mongoConn, remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd
from ast import literal_eval

In [59]:
pymysql.install_as_MySQLdb() 
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
alchemy_conn = engine.connect()

In [60]:
def generate_user_service_csv(): 
    df = pd.read_sql("SELECT Service_Id id , Service_Name SNAME From streamingservices", con=alchemy_conn)
    serviceDct={}
    for i, row in df.iterrows():
        serviceDct[row['SNAME'].replace(' ','_').upper()] = row['id']
    serviceDct['AMAZON']=37
    serviceDct['YOUTUBE']=80  
    
    Ndf = pd.read_csv("data/NIESEN_VOD_VIEWING_HABITS-V4.csv")
    Ndf = Ndf.loc[:, ['username', 'services', 'freq']]
    lst = []
    for i, row in Ndf.iterrows(): 
        service_dict = literal_eval(row.services) 
        for k, v in service_dict.items():
            k = k.upper().replace(' ','_')
            try:
                lst.append({
                    'username': row.username
                    , 'service_id':serviceDct[k]
                    , 'freq':row.freq.split('-')[0]
                })  
            except:
                pass
    df = pd.DataFrame(lst)
    df.to_csv('data/user_service.csv', index=False)
generate_user_service_csv()

In [61]:
df = pd.read_csv('data/user_service.csv')
df 

,username,service_id,freq
0,fabian_couture,30,1
1,winford_underwood,80,2
2,winford_underwood,12,2
3,darell_akers,30,1
4,don_macon,54,1
...,...,...,...
16573,rod_golding,34,1
16574,rod_golding,20,1
16575,dannie_corcoran,80,3
16576,henry_bishop,80,2


,username,service_id
0,fabian_couture,30
1,winford_underwood,80
2,winford_underwood,12
3,darell_akers,30
4,don_macon,54
...,...,...
16573,rod_golding,34
16574,rod_golding,20
16575,dannie_corcoran,80
16576,henry_bishop,80
